In [92]:
using Pkg
Pkg.activate("..")
using Dispersions
using Test
using FFTW
include("../test/helper_functions.jl")

  Activating project at `~/Codes/Dispersions.jl`


fcc_dispersion (generic function with 1 method)

In [95]:
Ns = 6


gr = "2Dsc-1.1"
kG = gen_kGrid(gr, Ns)

v_full = collect(Dispersions.gen_sampling(grid_type(kG), 2, Ns))
q0 = Tuple(findfirst(x-> all(x .≈ 0), v_full))

arr1 = randn(ComplexF64, gridshape(kG))
arr2 = randn(ComplexF64, gridshape(kG))
arr1_sym = expandKArr(kG, reduceKArr(kG, arr1))
arr2_sym = expandKArr(kG, reduceKArr(kG, arr2))
arr1_c = deepcopy(arr1_sym)
arr2_c = deepcopy(arr2_sym);

In [96]:
res_pp, dbg_i_pp, dbg_val_pp = naive_conv(v_full, v_full, pp=true, q0, round_entries=false);
res_ph, dbg_i, dbg_val = naive_conv(v_full, v_full, pp=false, q0, round_entries=false);

In [97]:
q_list = map(el->check_q(el, Ns), dbg_val);
# preprocess rounds all entries in order for unqiue to work
q_list_pre = map(x -> map(xi -> round.(xi,digits=8), x), q_list)
q_list_pre = map(x -> map(xi -> map(xii -> xii ≈ 0 ? abs(xii) : xii, xi), x), q_list_pre)
# Test if all entries have the same common q difference
@test all(map(qi_list -> length(unique(qi_list)) == 1, q_list_pre))
# Extract only q vector (since we checked that only one unique exists
q_list_single = map(x->x[1], q_list_pre)
# Test if all q-vectors are at the correct index
@test all(map(x-> all(x[1] .≈ x[2]), zip(q_list_single, v_full)))
# Test if all possible q-vectors are contained in convolution
@test all(map(el -> all(el[1] .≈ el[2]), zip(sort(unique(v_full)[:]), sort(map(qi->qi[1], q_list_pre)[:]))))
# Test if all combinations of k_i and k_{i+j} are contained in each result index
@test all(map(entry -> length(unique(map(x->x[1], entry))), dbg_val) .== Ns*Ns)
@test all(map(entry -> length(unique(map(x->x[2], entry))), dbg_val) .== Ns*Ns)

Test Passed

In [98]:
q_list_pp = map(el->check_q(el, Ns, pp=true), dbg_val_pp);
# preprocess rounds all entries in order for unqiue to work
q_list_pp_pre = map(x -> map(xi -> round.(xi,digits=8), x), q_list_pp)
q_list_pp_pre = map(x -> map(xi -> map(xii -> xii ≈ 0 ? abs(xii) : xii, xi), x), q_list_pp_pre)
# Test if all entries have the same common q difference
@test all(map(qi_list -> length(unique(qi_list)) == 1, q_list_pp_pre))
# extract only q vector (since we checked that only one unique exists
q_list_pp_single = map(x->x[1], q_list_pp_pre)
# test if all q-vectors are at the correct index
@test all(map(x-> all(x[1] .≈ x[2]), zip(q_list_pp_single, v_full)))
# Test if all possible q-vectors are contained in convolution
@test all(map(el -> all(el[1] .≈ el[2]), zip(sort(unique(v_full)[:]), sort(map(qi->qi[1], q_list_pp_pre)[:]))))
# Test if all combinations of k_i and k_{i+j} are contained in each result index
@test all(map(entry -> length(unique(map(x->x[1], entry))), dbg_val_pp) .== Ns*Ns)
@test all(map(entry -> length(unique(map(x->x[2], entry))), dbg_val_pp) .== Ns*Ns)

Test Failed at In[98]:6
  Expression: all(map((qi_list->begin
                length(unique(qi_list)) == 1
            end), q_list_pp_pre))



LoadError: [91mThere was an error during testing[39m

In [99]:
conv_ph_naive,_,_ = naive_conv(arr1_sym, arr2_sym, q0, pp=false);
conv_test = Dispersions.conv(kG, reduceKArr(kG,arr1_sym), reduceKArr(kG,arr2_sym), crosscorrelation=true)
conv_ph_fft_test = circshift(reverse(ifft(fft(arr1_sym) .* fft(reverse(arr2_sym)))), q0 .- 1)

show(stdout, "text/plain", expandKArr(kG, conv_test))
println("\n===============================")
show(stdout, "text/plain", kG.cache1 ./ kG.Nk)
println("\n===============================")
show(stdout, "text/plain", conv_ph_naive ./ kG.Nk)
println("\n===============================")
show(stdout, "text/plain", conv_ph_fft_test ./ kG.Nk)

@test all(conv_ph_fft_test .≈ conv_ph_naive)
@test all(conv_ph_fft_test .≈ expandKArr(kG, conv_test) .* kG.Nk)

6×6 Matrix{ComplexF64}:
 0.0435964-0.0257843im     0.137878+0.0708161im   0.0326768-0.00521829im    0.137878+0.0708161im  0.0435964-0.0257843im     0.104766+0.0685041im
  0.137878+0.0708161im      0.20175+0.150119im   0.00471098+0.0754758im      0.20175+0.150119im    0.137878+0.0708161im     0.123742+0.119448im
 0.0326768-0.00521829im  0.00471098+0.0754758im   0.0720781-0.0545361im   0.00471098+0.0754758im  0.0326768-0.00521829im  -0.0273605+0.0730325im
  0.137878+0.0708161im      0.20175+0.150119im   0.00471098+0.0754758im      0.20175+0.150119im    0.137878+0.0708161im     0.123742+0.119448im
 0.0435964-0.0257843im     0.137878+0.0708161im   0.0326768-0.00521829im    0.137878+0.0708161im  0.0435964-0.0257843im     0.104766+0.0685041im
  0.104766+0.0685041im     0.123742+0.119448im   -0.0273605+0.0730325im     0.123742+0.119448im    0.104766+0.0685041im     0.263201+0.226195im
6×6 Matrix{ComplexF64}:
    0.20175+0.150119im    0.137878+0.0708161im     0.123742+0.119448im    0.137878+0.

Test Passed

In [100]:
conv_pp_naive,_,_ = naive_conv(arr1_sym, arr2_sym, q0, pp=true);
conv_pp_test = Dispersions.conv(kG, reduceKArr(kG,arr1_sym), reduceKArr(kG, arr2_sym), crosscorrelation=false)
conv_pp_fft_test = circshift(ifft(fft(arr1_sym) .* fft(arr2_sym)), -0 .* q0 .+ 2)

#show(stdout, "text/plain", expandKArr(kG, conv_pp_test))
#println("\n===============================")
#show(stdout, "text/plain", kG.cache1 ./ kG.Nk)
#println("\n===============================")
show(stdout, "text/plain", conv_pp_naive  ./ kG.Nk )
println("\n===============================")
show(stdout, "text/plain", conv_pp_fft_test ./ kG.Nk)

@test all(conv_pp_fft_test .≈ conv_pp_naive)
@test all(conv_pp_fft_test ./ kG.Nk .≈ expandKArr(kG, conv_pp_test))

6×6 Matrix{ComplexF64}:
 0.0435964-0.0257843im     0.137878+0.0708161im   0.0326768-0.00521829im    0.137878+0.0708161im  0.0435964-0.0257843im     0.104766+0.0685041im
  0.137878+0.0708161im      0.20175+0.150119im   0.00471098+0.0754758im      0.20175+0.150119im    0.137878+0.0708161im     0.123742+0.119448im
 0.0326768-0.00521829im  0.00471098+0.0754758im   0.0720781-0.0545361im   0.00471098+0.0754758im  0.0326768-0.00521829im  -0.0273605+0.0730325im
  0.137878+0.0708161im      0.20175+0.150119im   0.00471098+0.0754758im      0.20175+0.150119im    0.137878+0.0708161im     0.123742+0.119448im
 0.0435964-0.0257843im     0.137878+0.0708161im   0.0326768-0.00521829im    0.137878+0.0708161im  0.0435964-0.0257843im     0.104766+0.0685041im
  0.104766+0.0685041im     0.123742+0.119448im   -0.0273605+0.0730325im     0.123742+0.119448im    0.104766+0.0685041im     0.263201+0.226195im
6×6 Matrix{ComplexF64}:
  0.0720781-0.0545361im   0.00471098+0.0754758im  0.0326768-0.00521829im  -0.0273605+

LoadError: [91mThere was an error during testing[39m

In [101]:
@test all(arr1_c .== arr1_sym)
@test all(arr2_c .== arr2_sym)

Test Passed

# k-k'

In [142]:
q = 5
k_sampling_full  = expandKArr(kG, gridPoints(kG))[:]
k_plus_q = map(k -> k_sampling_full[k] .+ q, 1:length(k_sampling_full))

k_plus_q2 = map(ki -> ki .+ q, expandKArr(kG, gridPoints(kG)))

6×6 Matrix{Tuple{Float64, Float64}}:
 (7.0944, 7.0944)   (7.0944, 6.0472)   (7.0944, 5.0)   …  (8.14159, 7.0944)
 (7.0944, 6.0472)   (6.0472, 6.0472)   (6.0472, 5.0)      (8.14159, 6.0472)
 (7.0944, 5.0)      (6.0472, 5.0)      (5.0, 5.0)         (8.14159, 5.0)
 (7.0944, 6.0472)   (6.0472, 6.0472)   (6.0472, 5.0)      (8.14159, 6.0472)
 (7.0944, 7.0944)   (7.0944, 6.0472)   (7.0944, 5.0)      (8.14159, 7.0944)
 (8.14159, 7.0944)  (8.14159, 6.0472)  (8.14159, 5.0)  …  (8.14159, 8.14159)

In [148]:
all(map(x-> all(x[1] .≈ x[2]) , zip(k_plus_q, k_plus_q2[:])))

true

In [ ]:
expandKArr(kG, gridPoints(kG)) .Q 

In [116]:
v_full
show(stdout, "text/plain", v_full)

6×6 Matrix{Tuple{Float64, Float64}}:
 (-2.0944, -2.0944)  (-2.0944, -1.0472)  (-2.0944, 0.0)  (-2.0944, 1.0472)  (-2.0944, 2.0944)  (-2.0944, 3.14159)
 (-1.0472, -2.0944)  (-1.0472, -1.0472)  (-1.0472, 0.0)  (-1.0472, 1.0472)  (-1.0472, 2.0944)  (-1.0472, 3.14159)
 (0.0, -2.0944)      (0.0, -1.0472)      (0.0, 0.0)      (0.0, 1.0472)      (0.0, 2.0944)      (0.0, 3.14159)
 (1.0472, -2.0944)   (1.0472, -1.0472)   (1.0472, 0.0)   (1.0472, 1.0472)   (1.0472, 2.0944)   (1.0472, 3.14159)
 (2.0944, -2.0944)   (2.0944, -1.0472)   (2.0944, 0.0)   (2.0944, 1.0472)   (2.0944, 2.0944)   (2.0944, 3.14159)
 (3.14159, -2.0944)  (3.14159, -1.0472)  (3.14159, 0.0)  (3.14159, 1.0472)  (3.14159, 2.0944)  (3.14159, 3.14159)

In [117]:
minus_v_full = map(x-> -1 .* x, v_full)
show(stdout, "text/plain", minus_v_full)

6×6 Matrix{Tuple{Float64, Float64}}:
 (2.0944, 2.0944)    (2.0944, 1.0472)    (2.0944, -0.0)    (2.0944, -1.0472)    (2.0944, -2.0944)    (2.0944, -3.14159)
 (1.0472, 2.0944)    (1.0472, 1.0472)    (1.0472, -0.0)    (1.0472, -1.0472)    (1.0472, -2.0944)    (1.0472, -3.14159)
 (-0.0, 2.0944)      (-0.0, 1.0472)      (-0.0, -0.0)      (-0.0, -1.0472)      (-0.0, -2.0944)      (-0.0, -3.14159)
 (-1.0472, 2.0944)   (-1.0472, 1.0472)   (-1.0472, -0.0)   (-1.0472, -1.0472)   (-1.0472, -2.0944)   (-1.0472, -3.14159)
 (-2.0944, 2.0944)   (-2.0944, 1.0472)   (-2.0944, -0.0)   (-2.0944, -1.0472)   (-2.0944, -2.0944)   (-2.0944, -3.14159)
 (-3.14159, 2.0944)  (-3.14159, 1.0472)  (-3.14159, -0.0)  (-3.14159, -1.0472)  (-3.14159, -2.0944)  (-3.14159, -3.14159)

In [138]:
shift_vec = 2 .* kG.k0 .- gridshape(kG) .- 1
minus_v_full2 = circshift(reverse(v_full), shift_vec)
show(stdout, "text/plain", minus_v_full2)

6×6 Matrix{Tuple{Float64, Float64}}:
 (2.0944, 2.0944)   (2.0944, 1.0472)   (2.0944, 0.0)   (2.0944, -1.0472)   (2.0944, -2.0944)   (2.0944, 3.14159)
 (1.0472, 2.0944)   (1.0472, 1.0472)   (1.0472, 0.0)   (1.0472, -1.0472)   (1.0472, -2.0944)   (1.0472, 3.14159)
 (0.0, 2.0944)      (0.0, 1.0472)      (0.0, 0.0)      (0.0, -1.0472)      (0.0, -2.0944)      (0.0, 3.14159)
 (-1.0472, 2.0944)  (-1.0472, 1.0472)  (-1.0472, 0.0)  (-1.0472, -1.0472)  (-1.0472, -2.0944)  (-1.0472, 3.14159)
 (-2.0944, 2.0944)  (-2.0944, 1.0472)  (-2.0944, 0.0)  (-2.0944, -1.0472)  (-2.0944, -2.0944)  (-2.0944, 3.14159)
 (3.14159, 2.0944)  (3.14159, 1.0472)  (3.14159, 0.0)  (3.14159, -1.0472)  (3.14159, -2.0944)  (3.14159, 3.14159)

(-1, -1)

In [134]:
gridshape(kG)

(6, 6)

In [132]:
kG.k0

(3, 3)

In [124]:
minus_v_full3 = reverse(circshift(v_full, -1 .* kG.k0 )) #circshift(reverse(v_full), 1 .* kG.k0 .- 1)
show(stdout, "text/plain", minus_v_full3)

6×6 Matrix{Tuple{Float64, Float64}}:
 (0.0, 0.0)      (0.0, -1.0472)      (0.0, -2.0944)      (0.0, 3.14159)      (0.0, 2.0944)      (0.0, 1.0472)
 (-1.0472, 0.0)  (-1.0472, -1.0472)  (-1.0472, -2.0944)  (-1.0472, 3.14159)  (-1.0472, 2.0944)  (-1.0472, 1.0472)
 (-2.0944, 0.0)  (-2.0944, -1.0472)  (-2.0944, -2.0944)  (-2.0944, 3.14159)  (-2.0944, 2.0944)  (-2.0944, 1.0472)
 (3.14159, 0.0)  (3.14159, -1.0472)  (3.14159, -2.0944)  (3.14159, 3.14159)  (3.14159, 2.0944)  (3.14159, 1.0472)
 (2.0944, 0.0)   (2.0944, -1.0472)   (2.0944, -2.0944)   (2.0944, 3.14159)   (2.0944, 2.0944)   (2.0944, 1.0472)
 (1.0472, 0.0)   (1.0472, -1.0472)   (1.0472, -2.0944)   (1.0472, 3.14159)   (1.0472, 2.0944)   (1.0472, 1.0472)

In [105]:
map(x-> x[1] .≈ x[2], zip(minus_v_full, minus_v_full2))

6×6 Matrix{Tuple{Bool, Bool}}:
 (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)
 (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)
 (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)
 (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)
 (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)
 (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)  (0, 0)

In [58]:

function build_k_minus_kp(arr)
    res = Array{eltype(arr),2}(undef, length(arr), length(arr))
    for (i,el_i) in enumerate(arr[:])
        for (j,el_j) in enumerate(arr[:])
            res[i,j] = el_i .- el_j
        end
    end
    res
end

function find_kGrid_slice(arr, kG_indices)
    res = falses(size(arr)...)
    for i in eachindex(arr)
        if arr[i] in kG_indices
            res[i] = true
        else
            false
        end
    end
    return res
end

find_kGrid_slice (generic function with 1 method)

In [59]:
res_test = build_k_minus_kp(v_full)
slice_test = find_kGrid_slice(res_test, v_full)
res_test

16×16 Matrix{Tuple{Float64, Float64}}:
 (0.0, 0.0)          (-1.5708, 0.0)      …  (-4.71239, -4.71239)
 (1.5708, 0.0)       (0.0, 0.0)             (-3.14159, -4.71239)
 (3.14159, 0.0)      (1.5708, 0.0)          (-1.5708, -4.71239)
 (4.71239, 0.0)      (3.14159, 0.0)         (0.0, -4.71239)
 (0.0, 1.5708)       (-1.5708, 1.5708)      (-4.71239, -3.14159)
 (1.5708, 1.5708)    (0.0, 1.5708)       …  (-3.14159, -3.14159)
 (3.14159, 1.5708)   (1.5708, 1.5708)       (-1.5708, -3.14159)
 (4.71239, 1.5708)   (3.14159, 1.5708)      (0.0, -3.14159)
 (0.0, 3.14159)      (-1.5708, 3.14159)     (-4.71239, -1.5708)
 (1.5708, 3.14159)   (0.0, 3.14159)         (-3.14159, -1.5708)
 (3.14159, 3.14159)  (1.5708, 3.14159)   …  (-1.5708, -1.5708)
 (4.71239, 3.14159)  (3.14159, 3.14159)     (0.0, -1.5708)
 (0.0, 4.71239)      (-1.5708, 4.71239)     (-4.71239, 0.0)
 (1.5708, 4.71239)   (0.0, 4.71239)         (-3.14159, 0.0)
 (3.14159, 4.71239)  (1.5708, 4.71239)      (-1.5708, 0.0)
 (4.71239, 4.71239)  (3.

In [60]:
count(slice_test)

144

In [64]:
circshift(reshape(unique(res_test[slice_test]), 4,4), (1,1))

4×4 Matrix{Tuple{Float64, Float64}}:
 (-1.5708, -1.5708)  (0.0, 1.5708)   (1.5708, 3.14159)  (-1.5708, 3.14159)
 (0.0, -1.5708)      (0.0, 0.0)      (1.5708, 1.5708)   (3.14159, 3.14159)
 (1.5708, -1.5708)   (1.5708, 0.0)   (3.14159, 1.5708)  (-1.5708, 0.0)
 (3.14159, -1.5708)  (3.14159, 0.0)  (0.0, 3.14159)     (-1.5708, 1.5708)

In [63]:
v_full

4×4 Matrix{Tuple{Float64, Float64}}:
 (-1.5708, -1.5708)  (-1.5708, 0.0)  (-1.5708, 1.5708)  (-1.5708, 3.14159)
 (0.0, -1.5708)      (0.0, 0.0)      (0.0, 1.5708)      (0.0, 3.14159)
 (1.5708, -1.5708)   (1.5708, 0.0)   (1.5708, 1.5708)   (1.5708, 3.14159)
 (3.14159, -1.5708)  (3.14159, 0.0)  (3.14159, 1.5708)  (3.14159, 3.14159)

In [38]:
unique(res_test)

343-element Vector{Tuple{Float64, Float64, Float64}}:
 (0.0, 0.0, 0.0)
 (1.5707963267948966, 0.0, 0.0)
 (3.141592653589793, 0.0, 0.0)
 (4.71238898038469, 0.0, 0.0)
 (0.0, 1.5707963267948966, 0.0)
 (1.5707963267948966, 1.5707963267948966, 0.0)
 (3.141592653589793, 1.5707963267948966, 0.0)
 (4.71238898038469, 1.5707963267948966, 0.0)
 (0.0, 3.141592653589793, 0.0)
 (1.5707963267948966, 3.141592653589793, 0.0)
 (3.141592653589793, 3.141592653589793, 0.0)
 (4.71238898038469, 3.141592653589793, 0.0)
 (0.0, 4.71238898038469, 0.0)
 ⋮
 (3.141592653589793, -3.141592653589793, -4.71238898038469)
 (4.71238898038469, -3.141592653589793, -4.71238898038469)
 (-1.5707963267948966, -3.141592653589793, -4.71238898038469)
 (-3.141592653589793, -3.141592653589793, -4.71238898038469)
 (-4.71238898038469, -3.141592653589793, -4.71238898038469)
 (0.0, -4.71238898038469, -4.71238898038469)
 (1.5707963267948966, -4.71238898038469, -4.71238898038469)
 (3.141592653589793, -4.71238898038469, -4.71238898038469)
 

In [22]:
unique(v_full)

64-element Vector{Tuple{Float64, Float64, Float64}}:
 (0.0, 0.0, 0.0)
 (1.5707963267948966, 0.0, 0.0)
 (3.141592653589793, 0.0, 0.0)
 (4.71238898038469, 0.0, 0.0)
 (0.0, 1.5707963267948966, 0.0)
 (1.5707963267948966, 1.5707963267948966, 0.0)
 (3.141592653589793, 1.5707963267948966, 0.0)
 (4.71238898038469, 1.5707963267948966, 0.0)
 (0.0, 3.141592653589793, 0.0)
 (1.5707963267948966, 3.141592653589793, 0.0)
 (3.141592653589793, 3.141592653589793, 0.0)
 (4.71238898038469, 3.141592653589793, 0.0)
 (0.0, 4.71238898038469, 0.0)
 ⋮
 (0.0, 1.5707963267948966, 4.71238898038469)
 (1.5707963267948966, 1.5707963267948966, 4.71238898038469)
 (3.141592653589793, 1.5707963267948966, 4.71238898038469)
 (4.71238898038469, 1.5707963267948966, 4.71238898038469)
 (0.0, 3.141592653589793, 4.71238898038469)
 (1.5707963267948966, 3.141592653589793, 4.71238898038469)
 (3.141592653589793, 3.141592653589793, 4.71238898038469)
 (4.71238898038469, 3.141592653589793, 4.71238898038469)
 (0.0, 4.71238898038469, 4.7